# SIADEX HTN ENGINE

imports requireds

In [1]:
!pip install ..
!pip install ../unified-planning

Processing /workspaces/up-siadex
  Preparing metadata (setup.py) ... done
  Created wheel for up-siadex: filename=up_siadex-0.0.1-py3-none-any.whl size=8664364 sha256=b4fb71b4c91d7d7baaf2c0f9feb6021d145b69ade0ba2b142c742245ec9c75a5
  Stored in directory: /home/codespace/.cache/pip/wheels/63/55/a1/26a2eb3a61793baa41bdcf5daf2fa5975374c48f8639a70b08
Successfully built up-siadex
  Attempting uninstall: up-siadex
    Found existing installation: up-siadex 0.0.1
    Uninstalling up-siadex-0.0.1:
      Successfully uninstalled up-siadex-0.0.1
Processing /workspaces/up-siadex/unified-planning
  Preparing metadata (setup.py) ... done
  Created wheel for unified-planning: filename=unified_planning-0.4.2-py3-none-any.whl size=356142 sha256=571e25da8d4b3b6d274329d7fd90b5b75b4a3e116781a723052eee3eb25287da
  Stored in directory: /home/codespace/.cache/pip/wheels/c4/05/f2/821c8767d2971ed17287b1008afb0607f41dad844571a5b40f
Successfully built unified-planning
  Attempting uninstall: unified-planning
  

In [2]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus


## Registering the engine

In order to use `SIADEX`, we need to register it among the set of planning engines available for the UP library as follows.

In [3]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

## Reading a problem from a file (HDDL)

In [4]:
reader = PDDLReader()
problem = reader.parse_problem("../unified-planning/unified_planning/test/pddl/htn-transport/domain.hddl", "../unified-planning/unified_planning/test/pddl/htn-transport/problem.hddl")

## Solving the problem with siadex

In [5]:
with OneshotPlanner(name='siadex') as p:
    result = p.solve(problem)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name()} found a valid plan!')
        print(f'The plan is: \n')
        for i,a in enumerate(result.plan.actions):
            print(f"{i}: {a}")
    else:
        print('No plan found!')

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_env().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_13835/140634004.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

SIADEX found a valid plan!
The plan is: 

0: drive(truck-0, city-loc-2, city-loc-1)
1: pick-up(truck-0, city-loc-1, package-0, capacity-0, capacity-1)
2: drive(truck-0, city-loc-1, city-loc-0)
3: drop(truck-0, city-loc-0, package-0, capacity-0, capacity-1)
4: drive(truck-0, city-loc-0, city-loc-1)
5: pick-up(truck-0, city-loc-1, package-1, capacity-0, capacity-1)
6: drive(truck-0, city-loc-1, city-loc-0)
7: drop(truck-0, city-loc-0, package-1, capacity-0, capacity-1)


/home/codespace/.python/current/lib/python3.10/site-packages/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7fa1767e5360> cannot solve this kind of problem!
  warn(msg)


In [6]:
siadex = OneshotPlanner(name='siadex')
debugger = siadex.debugger()

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_13835/1745171064.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE



In [7]:
debugger.debug(problem)

In [9]:
debugger.state()

[road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

In [15]:
debugger.force_run("next")

debug:> next
__________________________________________________
(*** 1 ***) Expanding: [0] (deliver package_0 city_loc_0)

(*** 1 ***) Selecting a candidate task.
(*** 1 ***) Found: 1 candidates (left).
      [0] :task (deliver ?p ?l)


In [16]:
debugger.state()

[road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

In [17]:
debugger.force_run("help print")

debug:> help print
__________________________________________________
Command: `print'. Shortcut: `p'

Description: Prints information about the planning context.
	`print state':	Prints the current state.
	`print agenda':	Prints the current agenda.
	`print plan':	Prints the ongoing plan.
	`print options':	Prints the options you can peform on next step.
	`print termtable':	Prints the internal term table.
	`print tasks':	Prints all the tasks defined in the domain.
	`print predicates':	Prints all the avaliable predicates defined in the domain.
	`print <predicate-expression>':	Prints all the predicates in the current state, or the tasks in current plan, that match the given expression.


In [14]:
debugger.force_run("help next")

debug:> help next
__________________________________________________
Command: `next'. ShortCut: `n'

Description: Advance one more step.
See also: `continue'


In [10]:
debugger.stop()

### END DEBUGGER

In [6]:
def solve_with_siadex(problem):
    with OneshotPlanner(name='siadex') as p:
        result = p.solve(problem)
        print(result.log_messages)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{p.name()} found a valid plan!')
            print(f'The plan is: \n')
            for i,a in enumerate(result.plan.actions):
                print(f"{i}: {a}")
            return result.plan
        else:
            return []
            print('No plan found!')

## Example of a hierarchical  problem done with UPF

In [7]:
htn = HierarchicalProblem()

#_________Objects_________#
Location = UserType("Location")

l1 = htn.add_object("l1", Location)
l2 = htn.add_object("l2", Location)
l3 = htn.add_object("l3", Location)
l4 = htn.add_object("l4", Location)

#_________Fluents_________#
loc = Fluent("is_on", l=Location)
htn.add_fluent(loc, default_initial_value=False)

connected = Fluent("connected", l1=Location, l2=Location)
htn.add_fluent(connected, default_initial_value=False)
htn.set_initial_value(connected(l1, l2), True)
htn.set_initial_value(connected(l2, l3), True)
htn.set_initial_value(connected(l3, l4), True)
htn.set_initial_value(connected(l4, l3), True)
htn.set_initial_value(connected(l3, l2), True)
htn.set_initial_value(connected(l2, l1), True)
htn.set_initial_value(loc(l1), True)


#_________Actions_________#
move = InstantaneousAction("move", l_from=Location, l_to=Location)
l_from = move.parameter("l_from")
l_to = move.parameter("l_to")
move.add_precondition(loc(l_from))
move.add_precondition(connected(l_from, l_to))
move.add_effect(loc(l_from), False)
move.add_effect(loc(l_to), True)
htn.add_action(move)

#_________Task_________#
go = htn.add_task("go", target=Location)

#_________Methods_________#
go_noop = Method("go-noop", target=Location)
go_noop.set_task(go)
target = go_noop.parameter("target")
go_noop.add_precondition(loc(target))

htn.add_method(go_noop)

go_recursive = Method(
    "go-recursive", source=Location, inter=Location, target=Location
)

go_recursive.set_task(go, go_recursive.parameter("target"))

source = go_recursive.parameter("source")
inter = go_recursive.parameter("inter")
target = go_recursive.parameter("target")

go_recursive.add_precondition(loc(source))
go_recursive.add_precondition(connected(source, inter))

t1 = go_recursive.add_subtask(move, source, inter)
t2 = go_recursive.add_subtask(go, target)
go_recursive.set_ordered(t1, t2)
htn.add_method(go_recursive)


#_________Init_________#
go1 = htn.task_network.add_subtask(go, l4)

In [8]:
solve_with_siadex(htn)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_20228/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: move(l1, l2)
1: move(l2, l3)
2: move(l3, l4)


[move(l1, l2), move(l2, l3), move(l3, l4)]

## Reading a problem from a file (HDPL)

In [9]:
reader = HPDLReader()
miconic = reader.parse_problem("../examples/ipc/Miconic/domain.hpdl", "../examples/ipc/Miconic/problem.hpdl")
rover = reader.parse_problem("../examples/ipc/Rover/domain.hpdl","../examples/ipc/Rover/problem.hpdl")
satellite = reader.parse_problem("../examples/ipc/Satellite/domain.hpdl","../examples/ipc/Satellite/problem.hpdl")
# smartphone = reader.parse_problem("../examples/ipc/SmartPhone/domain.hpdl","../examples/ipc/SmartPhone/problem.hpdl")
transport = reader.parse_problem("../examples/ipc/Transport/domain.hpdl","../examples/ipc/Transport/problem.hpdl")
# translog = reader.parse_problem("../examples/ipc/UM-Translog/domain.hpdl","../examples/ipc/UM-Translog/problem.hpdl")
zeno = reader.parse_problem("../examples/ipc/Zenotravel/domain.hpdl","../examples/ipc/Zenotravel/problem.hpdl")

In [10]:
solve_with_siadex(rover)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_20228/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n\n[Error]: Empty stack\n')]
SIADEX found a valid plan!
The plan is: 



[]

In [11]:
solve_with_siadex(transport)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_20228/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: drive_primitive(truck_0, city_loc_2, city_loc_1)
1: pick_up_primitive(truck_0, city_loc_1, package_0, capacity_0, capacity_1)
2: drive_primitive(truck_0, city_loc_1, city_loc_0)
3: drop_primitive(truck_0, city_loc_0, package_0, capacity_0, capacity_1)
4: drive_primitive(truck_0, city_loc_0, city_loc_1)
5: pick_up_primitive(truck_0, city_loc_1, package_1, capacity_0, capacity_1)
6: drive_primitive(truck_0, city_loc_1, city_loc_2)
7: drop_primitive(truck_0, city_loc_2, package_1, capacity_0, capacity_1)


[drive_primitive(truck_0, city_loc_2, city_loc_1), pick_up_primitive(truck_0, city_loc_1, package_0, capacity_0, capacity_1), drive_primitive(truck_0, city_loc_1, city_loc_0), drop_primitive(truck_0, city_loc_0, package_0, capacity_0, capacity_1), drive_primitive(truck_0, city_loc_0, city_loc_1), pick_up_primitive(truck_0, city_loc_1, package_1, capacity_0, capacity_1), drive_primitive(truck_0, city_loc_1, city_loc_2), drop_primitive(truck_0, city_loc_2, package_1, capacity_0, capacity_1)]

In [16]:
solve_with_siadex(zeno)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_6645/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: fly_primitive(a1, c2, c4, f6, f5)
1: board_primitive(p1, a1, c4)
2: fly_primitive(a1, c4, c3, f5, f4)
3: debark_primitive(p1, a1, c3)
4: fly_primitive(a1, c3, c2, f4, f3)
5: board_primitive(p4, a1, c2)
6: fly_primitive(a1, c2, c4, f3, f2)
7: debark_primitive(p4, a1, c4)
8: fly_primitive(a1, c4, c1, f2, f1)
9: board_primitive(p3, a1, c1)
10: fly_primitive(a1, c1, c4, f1, f0)
11: debark_primitive(p3, a1, c4)
12: refuel_primitive(a1, f0, f2)
13: fly_primitive(a1, c4, c1, f2, f0)
14: board_primitive(p2, a1, c1)
15: refuel_primitive(a1, f0, f2)
16: fly_primitive(a1, c1, c4, f2, f0)
17: debark_primitive(p2, a1

[fly_primitive(a1, c2, c4, f6, f5), board_primitive(p1, a1, c4), fly_primitive(a1, c4, c3, f5, f4), debark_primitive(p1, a1, c3), fly_primitive(a1, c3, c2, f4, f3), board_primitive(p4, a1, c2), fly_primitive(a1, c2, c4, f3, f2), debark_primitive(p4, a1, c4), fly_primitive(a1, c4, c1, f2, f1), board_primitive(p3, a1, c1), fly_primitive(a1, c1, c4, f1, f0), debark_primitive(p3, a1, c4), refuel_primitive(a1, f0, f2), fly_primitive(a1, c4, c1, f2, f0), board_primitive(p2, a1, c1), refuel_primitive(a1, f0, f2), fly_primitive(a1, c1, c4, f2, f0), debark_primitive(p2, a1, c4), goal_action_primitive]

In [12]:
solve_with_siadex(miconic)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_20228/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: move_primitive(f0, f0)
1: board_primitive(p0, f0)
2: move_primitive(f0, f1)
3: debark_primitive(p0, f1)
4: move_primitive(f1, f3)
5: board_primitive(p2, f3)
6: move_primitive(f3, f1)
7: debark_primitive(p2, f1)
8: move_primitive(f1, f2)
9: board_primitive(p4, f2)
10: move_primitive(f2, f1)
11: debark_primitive(p4, f1)
12: move_primitive(f1, f0)
13: board_primitive(p1, f0)
14: move_primitive(f0, f3)
15: debark_primitive(p1, f3)
16: move_primitive(f3, f3)
17: board_primitive(p3, f3)
18: move_primitive(f3, f2)
19: debark_primitive(p3, f2)


[move_primitive(f0, f0), board_primitive(p0, f0), move_primitive(f0, f1), debark_primitive(p0, f1), move_primitive(f1, f3), board_primitive(p2, f3), move_primitive(f3, f1), debark_primitive(p2, f1), move_primitive(f1, f2), board_primitive(p4, f2), move_primitive(f2, f1), debark_primitive(p4, f1), move_primitive(f1, f0), board_primitive(p1, f0), move_primitive(f0, f3), debark_primitive(p1, f3), move_primitive(f3, f3), board_primitive(p3, f3), move_primitive(f3, f2), debark_primitive(p3, f2)]

Bad pipe message: %s [b'\xe6\xd6\xc2\xbe:\x87\xf54\xeeiR\x04\xf7\x82\xee\xe7eh \xa9\xb3\xa1,\x90a\x1d\xe8\x02\xdf\xc3\x84>\xfe"\xab\xdd\xa3\x90I;=\xb8%\xba', b'\xe8\xbf.\xd0m\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05']
Bad pipe message: %s [b'\x03\x08']
Bad pipe message: %s [b'\x08\x08\t\x08\n\x08']
Bad pipe message: %s [b'\xca']
Bad pipe message: %s [b'\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x9b\x91+z\xfa\xfe\xca\xad\xe7\xf9/\x01\xb35\x9b\xa1\x1e\x9d\xd4gV!']
Bad pipe message: %s [b"\xdc\xc8\x8e\xf7\xad+\xba\xd6~\xd7\x91\x81\xde\x0fZ\x92tU\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\x